# 批次文件轉換 (Batch Conversion)

在前一節中，你已經學會如何用 Docling Python 函式庫程式化地轉換單一 PDF 檔案。
在實務案例中，通常需要一次轉換多個文件。Docling 提供了一個方便的方法：
`DocumentConverter` 類別中的 `convert_all` 方法，可以批次處理多個文件。

此方法會接收一個指向多個檔案的 `Path` 物件列表，並允許你控制錯誤處理、頁面大小等選項。


## 先決條件 (Pre-requisites)

- 已安裝 Docling Python 函式庫（建議於虛擬環境中使用 pip 安裝）
- Git CLI（用來從 GitHub 下載範例檔案）
- Visual Studio Code 或其他編輯器（用於查看與編輯程式碼）


In [1]:
# 下載範例資料與程式碼
!git clone https://github.com/RedHatQuickCourses/genai-apps.git


fatal: destination path 'genai-apps' already exists and is not an empty directory.


In [2]:
# 切換到包含範例資料的資料夾
%cd genai-apps/dataprep


/work/genai-apps/dataprep


## 批次轉換程式說明

在 `batch-conversion.py` 中：
- **輸入資料夾**：`sample-data/batch`
- **輸出資料夾**：`/tmp/batch-converted`

所有要轉換的 PDF 檔案放在輸入資料夾，轉換結果 (Markdown) 會輸出到 `/tmp/batch-converted`。


In [4]:
from pathlib import Path

INPUT_DIR = Path("sample-data/batch")
OUTPUT_DIR = Path("/tmp/batch-converted")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 取得待轉換的 PDF 清單
input_doc_paths = list(INPUT_DIR.glob("*.pdf"))
print("找到 PDF 檔數量：", len(input_doc_paths))
for p in input_doc_paths[:5]:
    print(" -", p.name)


找到 PDF 檔數量： 4
 - 2206.01062.pdf
 - redp5110_sampled.pdf
 - 2305.03393v1-pg9.pdf
 - 2203.01017v2.pdf


## 設定 PDF 處理選項

Docling 針對 PDF 提供預設良好設定（`PdfPipelineOptions`）。  
此處示範啟用「為每頁產生影像（於記憶體）」供後續流程使用。


```python
from docling.pipeline.standard_pdf_pipeline import PdfPipelineOptions

pipeline_options = PdfPipelineOptions()
pipeline_options.generate_page_images = True


## 建立轉換器（DocumentConverter）

這裡不手動指定 backend，使用 Docling 的預設（相容性較佳）。  
若需要自訂 backend，可在 `PdfFormatOption` 中加入 `backend=...`。


```python
from docling.document_converter import DocumentConverter, InputFormat, PdfFormatOption

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options
        )
    }
)
doc_converter


## 批次轉換

使用 `convert_all` 一次處理多個文件。  
這裡設定 `raises_on_error=False`，代表遇到錯誤不會中止整個批次流程。


```python
conv_results = doc_converter.convert_all(
    input_doc_paths,
    raises_on_error=False,   # 發生錯誤時不中斷整批
)
len(conv_results)


## 匯出為 Markdown

將每一個 `DoclingDocument`（記憶體表示）輸出為 `.md` 檔。  
可使用 VS Code 或其他 Markdown 工具預覽轉換結果。


```python
from docling.models.docling_document import ImageRefMode

saved = []
for conv_res in conv_results:
    if conv_res.document is None:
        # 例如該檔案轉換失敗或被略過
        print(f"[Skip] {conv_res.input_path.name}")
        continue

    out_md = OUTPUT_DIR / f"{conv_res.input_path.stem}.md"
    conv_res.document.save_as_markdown(out_md, image_mode=ImageRefMode.PLACEHOLDER)
    saved.append(out_md)

print(f"完成輸出 {len(saved)} 個 Markdown 檔案。")
for p in saved[:5]:
    print(" -", p)


## 檢視輸出結果

在 `/tmp/batch-converted` 應能看到與輸入檔名相對應的 `.md` 檔。  
開啟檢視標題、段落、表格是否正確轉換。


```python
for p in sorted(OUTPUT_DIR.glob("*.md"))[:10]:
    print(p.name)
print("總數：", len(list(OUTPUT_DIR.glob("*.md"))))


### 小提醒

- 若 PDF 為掃描影像且需要 OCR，請確認系統/容器中已安裝對應 OCR 引擎（如 Tesseract/EasyOCR），並按需求在 Docling 流程中啟用。
- 若不需要 OCR（PDF 本身有文字層），關閉 OCR 會更快更穩。
- 批次處理大量檔案時，請留意記憶體使用量（在容器/VM 中可調整分配）。


## 執行轉換程式

在 Notebook 或終端中執行：


In [32]:
!pip install torch torchvision torchaudio easyocr 


In [35]:
!python3 /work/genai-apps/dataprep/batch-conversion.py


2025-10-07 08:13:17,269 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-07 08:13:17,277 - INFO - Going to convert document batch...
2025-10-07 08:13:17,277 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 58627ba972cff869acb44714af27a59e
2025-10-07 08:13:17,279 - INFO - Loading plugin 'docling_defaults'
2025-10-07 08:13:17,279 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-07 08:13:17,281 - INFO - Loading plugin 'docling_defaults'
2025-10-07 08:13:17,282 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-07 08:13:17,284 - INFO - Accelerator device: 'cpu'
2025-10-07 08:13:17,286 - INFO - Accelerator device: 'cpu'
2025-10-07 08:13:18,592 - INFO - Accelerator device: 'cpu'
2025-10-07 08:13:18,746 - INFO - Processing document 2206.01062.pdf
2025-10-07 08:13:47,217 - INFO - Finished converting document 2206.01062.pdf in 29.95 sec.
2025-10-07 08:13:47,236 - INFO - detected formats: [<Inp

程式執行完成後，你應該會在 `/tmp/batch-converted` 資料夾看到對應的 `.md` 檔案。
可以用 VS Code 或 Markdown 編輯器打開檢視，確認標題、段落、表格等內容是否正確轉換。
